In [1]:
import numpy as np
import hoomd, hoomd.md


def lj(r, rmin, rmax, epsilon, sigma):
    V = 4 * epsilon * ( (sigma / r)**12 - (sigma / r)**6);
    F = 4 * epsilon / r * ( 12 * (sigma / r)**12 - 6 * (sigma / r)**6);
    return (V, F)

def jf(r, rmin, rmax, epsilon, a, A0, A1, A2, B0, B1, B2):
    V = epsilon * ( (a / r)**12 + A0/(1+np.exp(A1/A0*r/(a-A2))) - B0/(1+np.exp(B1/B0*r/(a-B2))))
    F = epsilon * ( -12*(a / r)**12/r - A1*np.exp(A1/A0*r/(a-A2))/(a-A2)/(1+np.exp(A1/A0*r/(a-A2)))**2 +
                   B1*np.exp(B1/B0*r/(a-B2))/(a-B2)/(1+np.exp(B1/B0*r/(a-B2)))**2)
    return (V, F)



hoomd.context.initialize()
unitcell=hoomd.lattice.sc(a=2.0, mass=1.0, type_name='A')
hoomd.init.create_lattice(unitcell=unitcell, n=10)
nl = hoomd.md.nlist.cell()

Usage: ipykernel_launcher.py [options]

ipykernel_launcher.py: error: no such option: -f


SystemExit: 2

/Users/user/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
table = hoomd.md.pair.table(width=1000, nlist=nl)
table.pair_coeff.set('A', 'A', func=lj, rmin=0.8, rmax=3.0, 
                     coeff=dict(epsilon=1.5, a=1.0, A0=1.0, A1=1.0, A2=0.5, B0=1.0, B1=1.0, B2=0.5))

all = hoomd.group.all();
d = hoomd.dump.gsd("mdFJ.gsd", period=1000, group=hoomd.group.all(), overwrite=True);
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.langevin(group=all, kT=1.2, seed=4)

In [ ]:
hoomd.run(10e3)

In [ ]:
tr = ex_render.get_trajectory("trajectory.gsd")
len(tr)